In [1]:
import os

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass, AssetExchange

from dotenv import load_dotenv

import pandas as pd

In [2]:
_ = load_dotenv()

### Access Alpaca Account

In [3]:
trading_client = TradingClient(os.environ["ALPACA_PAPER_KEY"], os.environ["ALPACA_PAPER_SECRET_KEY"])

In [4]:
account = trading_client.get_account()

In [5]:
account.cash

'100000'

### Work with Assets

Available asset classes

In [6]:
dir(AssetClass)[:4]

['CRYPTO', 'CRYPTO_PERP', 'US_EQUITY', 'US_OPTION']

In [14]:
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)

assets = trading_client.get_all_assets(search_params)
assets = [asset.model_dump() for asset in assets]
assets_df = pd.DataFrame(assets)
print(f"There are {len(assets_df)} assets on Alpaca...")
assets_df = assets_df[assets_df.tradable]
print(f"...of which {len(assets_df)} assets are tradable")


There are 32520 assets on Alpaca...
...of which 12322 assets are tradable


In [16]:
len(assets_df[(assets_df.exchange == AssetExchange.NASDAQ) | (assets_df.exchange == AssetExchange.NYSE)])

8070

In [42]:
symbols = assets_df[(assets_df.exchange == AssetExchange.NYSE) | (assets_df.exchange == AssetExchange.NASDAQ)].symbol.unique()

### Market Data

In [50]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

from datetime import datetime

In [69]:
client = StockHistoricalDataClient(os.environ["ALPACA_PAPER_KEY"], os.environ["ALPACA_PAPER_SECRET_KEY"])

request_params = StockBarsRequest(
    symbol_or_symbols=list(symbols),
    timeframe=TimeFrame.Day,
    start=datetime(2025, 12, 30),
    end=datetime(2025, 12, 31)
)

stock_bars = client.get_stock_bars(request_params)
stock_bars.df

,,open,high,low,close,volume,trade_count,vwap
symbol,timestamp,,,,,,,
BRCB,2025-12-30 05:00:00+00:00,23.090,24.15,22.7950,22.9700,591100.0,7033.0,23.054245
KNOP,2025-12-30 05:00:00+00:00,10.450,10.48,10.3236,10.3500,68164.0,1176.0,10.391824
NIO,2025-12-30 05:00:00+00:00,5.540,5.79,5.4950,5.5000,80148010.0,123374.0,5.618497
OBIO,2025-12-30 05:00:00+00:00,4.210,4.25,3.9700,4.1400,248552.0,3585.0,4.113836
QQQY,2025-12-30 05:00:00+00:00,23.860,23.93,23.8300,23.8900,138291.0,1749.0,23.899401
...,...,...,...,...,...,...,...,...
CPNG,2025-12-30 05:00:00+00:00,24.415,24.52,23.7200,24.1300,14709748.0,89406.0,24.044957
EMN,2025-12-30 05:00:00+00:00,63.570,64.44,63.4750,64.3300,838113.0,17475.0,64.146194
GDV.PRH,2025-12-30 05:00:00+00:00,22.450,22.45,22.3100,22.3143,8614.0,81.0,22.374087


In [64]:
quantiles = stock_bars.df.volume.quantile([0.1*i for i in range(0, 11)]).values
quantiles

array([0.0000000e+00, 1.7772000e+03, 1.0932200e+04, 3.4062200e+04,
       7.8946200e+04, 1.4617900e+05, 2.7153240e+05, 5.1191840e+05,
       9.4147340e+05, 2.0708772e+06, 8.3301512e+08])

In [67]:
stock_bars.df[stock_bars.df.volume > quantiles[-2]]

,,open,high,low,close,volume,trade_count,vwap
symbol,timestamp,,,,,,,
SOC,2025-12-30 05:00:00+00:00,8.680,9.300,8.355,8.7500,8674427.0,57057.0,8.655936
RF,2025-12-30 05:00:00+00:00,27.630,27.630,27.310,27.3200,5142866.0,33603.0,27.378820
KSS,2025-12-30 05:00:00+00:00,20.790,20.930,20.250,20.3500,3322397.0,34322.0,20.442336
BLNK,2025-12-30 05:00:00+00:00,0.660,0.675,0.630,0.6442,5954595.0,11452.0,0.643438
RZLT,2025-12-30 05:00:00+00:00,2.410,2.410,2.160,2.1900,5951046.0,24302.0,2.239108
...,...,...,...,...,...,...,...,...
AEG,2025-12-30 05:00:00+00:00,7.770,7.780,7.750,7.7700,2515673.0,5268.0,7.765386
GMAB,2025-12-30 05:00:00+00:00,31.980,32.000,31.395,31.4900,2461438.0,23004.0,31.666983
BW,2025-12-30 05:00:00+00:00,5.600,6.420,5.580,6.2700,4464606.0,25929.0,6.154198


In [76]:
stock_bars.df.loc["AAPL"]

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2025-12-30 05:00:00+00:00,272.81,274.08,272.28,273.08,22139617.0,431190.0,273.04204
